<a href="https://colab.research.google.com/github/liorZucker11/cloud-computing/blob/main/Shablool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##imports and pips:

In [47]:
!pip install ipywidgets
!pip install IPython
!pip install firebase
from IPython.display import display, HTML
import ipywidgets as widgets
from firebase import firebase
from nltk.stem import PorterStemmer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.9 MB/s eta 0:00:00


##firebase DB:

In [48]:
firebase = firebase.FirebaseApplication('https://cloudproject-shablool-default-rtdb.europe-west1.firebasedatabase.app/', None)

"""
  Retrieves 'links' and 'links_counter' data associated with the given key from Firebase index.
  Returns a transposed list or None if the key doesn't exist.
"""
def getKey(key):
  word_data = firebase.get(f'/index/{key}',None)
  if word_data == None:
    return word_data
  tmp = [word_data['links'],word_data['links_counter']]
  transposed = [[row[i] for row in tmp] for i in range(len(tmp[0]))]
  return transposed

## Functions for searching algorithem:

In [49]:
#first level
def sort_links_by_counter(links):
    # Sort the list of lists based on the second element (counter) in each sublist
    sorted_links = sorted(links, key=lambda x: x[1], reverse=True)
    # Extract the first element (link) from each sublist and return as a 1-dimensional list
    return [link[0] for link in sorted_links]


#second level
def filter_common_links(big_data_structure):
    # Step 1: Find common links across all wordDatas
    common_links = set(link for link, _ in big_data_structure[0])  # Initialize with links from the first wordData
    for wordData in big_data_structure[1:]:  # Start from the second wordData
        current_links = set(link for link, _ in wordData)
        common_links.intersection_update(current_links)  # Keep only links that are also in the current wordData
    # Step 2: Filter each wordData to keep only elements with links in common_links
    filtered_data_structure = []
    for wordData in big_data_structure:
        filtered_wordData = [[link, counter] for link, counter in wordData if link in common_links]
        filtered_data_structure.append(filtered_wordData)
    return filtered_data_structure


#third level
def calculate_average_counters(filtered_data_structure):
    link_counters = {}  # Dictionary to hold the total counters for each link and the number of occurrences
    # Aggregate counters for each link
    for wordData in filtered_data_structure:
        for link, counter in wordData:
            if link in link_counters:
                link_counters[link][0] += counter  # Add to the total counter
                link_counters[link][1] += 1  # Increment occurrences
            else:
                link_counters[link] = [counter, 1]  # Initialize with the first counter and set occurrences to 1
    # Calculate average for each link and prepare the result list
    average_counters = [[link, total_counter / occurrences] for link, (total_counter, occurrences) in link_counters.items()]
    # Sort the result list by link for consistent output, if needed
    average_counters.sort(key=lambda x: x[0])
    return average_counters

#fourth level
def sort_by_counters_desc(average_counters):
    # Sort the list by the counter value in descending order
    sorted_average_counters = sorted(average_counters, key=lambda x: x[1], reverse=True)
    return sorted_average_counters

#fivth level
def extract_links(average_counters):
    links_only = [link for link, _ in average_counters]
    return links_only

"""
  Searches for links associated with the stemmed words in the query.

  This function takes a search query as input, stems the words using Porter Stemmer,
  retrieves data for each stemmed word from the Firebase index using the getKey function,
  filters common links across all search results, calculates the average counters for each link,
  sorts the links by their counters in descending order, and finally returns a list of links
  extracted from the sorted results.

  If no results are found for the query, it returns a message indicating that the key
  does not exist in the index.
"""
def search(query):
    stemmer = PorterStemmer()
    query = query.lower()
    results = []
    query_words = query.split()
    for word in query_words:
      word = stemmer.stem(word)
      results.append(getKey(word))
    if results == []:
      return "The key does not exist in the index."
    print(results)
    level1 = filter_common_links(results)
    level2 = calculate_average_counters(level1)
    level3 = sort_by_counters_desc(level2)
    level4 = extract_links(level3)
    return level4

#Screens

In [50]:
#Initial Global Variables
container=widgets.Box()
results_output=widgets.Output()

##Admin Screen:

In [51]:
def create_admin_screen():
    # Function to handle return button click
    def on_return_button_clicked(b):
        container.children = []
        container.layout = widgets.Layout()
        Main_Screen()

    # Actions for buttons
    def on_delete_button_clicked(b):
        # Code to handle delete action
        pass

    def on_edit_button_clicked(b):
        # Code to handle edit action
        pass

    def on_add_button_clicked(b):
        # Code to handle add action
        pass

    def on_get_button_clicked(b):
        edit_button.disabled = False
        results_text.value = "Fetched data from the database..."
    # Input text box
    input_text = widgets.Text(
        placeholder='Enter text here...',
        description='Word:   ',
        disabled=False,
        layout=widgets.Layout(width='100%', margin='0 0 10px 0'),
        style={'description_width': 'initial', 'border': '1px solid black'}
    )

    # Larger text area for displaying results
    results_text = widgets.Textarea(
        placeholder='Results will be shown here...',
        description='Results:',
        disabled=False,
        layout=widgets.Layout(width='100%', height='200px', margin='0 0 10px 0'),
        style={'description_width': 'initial', 'border': '1px solid black'}
    )

    # Buttons for database operations
    delete_button = widgets.Button(description="Delete", button_style='danger', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
    edit_button = widgets.Button(description="Edit", disabled=True, layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
    add_button = widgets.Button(description="Add", button_style='success', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
    get_button = widgets.Button(description="Get", button_style='info', layout=widgets.Layout(width='100px', margin='0 5px 5px 0'))
    return_button = widgets.Button(description="Return", button_style='warning', layout=widgets.Layout(border_radius='50%', width='80px', height='50px'))


    # Attach actions to buttons
    delete_button.on_click(on_delete_button_clicked)
    edit_button.on_click(on_edit_button_clicked)
    add_button.on_click(on_add_button_clicked)
    get_button.on_click(on_get_button_clicked)
    return_button.on_click(on_return_button_clicked)

    # Arrange buttons horizontally
    buttons_hbox = widgets.HBox([get_button, add_button, edit_button, delete_button], layout=widgets.Layout(justify_content='flex-start'))

    # Logo
    with open("/content/logo.jpg", "rb") as file:
        image = file.read()
    logo = widgets.Image(value=image, format='jpg', layout=widgets.Layout(height='100px', width='auto'))

    # Arrange all widgets vertically
    additional_string = widgets.Label("Admin Screen")
    hBox_layout = widgets.Layout(display='flex', justify_content='space-between', align_items='center', width='70%', padding='10px', border='1px solid #ccc')
    hBox = widgets.HBox(children=[return_button,additional_string ,logo], layout=hBox_layout)

    vBox_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='70%', padding='10px', border='1px solid #ccc')
    vBox = widgets.VBox([input_text, results_text, buttons_hbox], layout=vBox_layout)

    container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', justify_content='flex-start', height='100vh')
    container = widgets.Box(children=[hBox, vBox], layout=container_layout)

    # Display everything
    display(container)


##Statistics Screen:

In [52]:
def create_statistics_screen():

  def on_return_button_clicked(b):
    container.children = []
    container.layout = widgets.Layout()
    Main_Screen()

  most_Freq = widgets.Label("Most Frequent Word: Azure")
  less_Freq = widgets.Label("Less Frequent Word: Play")
  most_search_Freq = widgets.Label("Most Search Frequent Word: Azure")

  input_text = widgets.Text(
    placeholder='If you want to know more enter a word...',
    description='Word:',
    disabled=False,
    layout=widgets.Layout(width='95%', margin='10px 10px 10px 10px'),
    style={'description_width': 'initial', 'border': '1px solid black'}
  )

    # Larger text area for displaying results
  results_text = widgets.Textarea(
      placeholder='Results will be shown here...',
      description='Result:',
      disabled=False,
      layout=widgets.Layout(width='100%', height='200px', margin='0 0 10px 0'),
      style={'description_width': 'initial', 'border': '1px solid black'}
  )

  return_button = widgets.Button(description="Return", button_style='warning', layout=widgets.Layout(border_radius='50%', width='80px', height='50px', alignment = 'left'))
  return_button.on_click(on_return_button_clicked)

  with open("/content/logo.jpg", "rb") as file:  # Update the path to your logo
      image = file.read()
  logo = widgets.Image(value=image, format='jpg', layout=widgets.Layout(height='100px', width='10%', margin = '0 0 20px 45%', alignment = 'center'))


  vbox_layout = widgets.Layout(display='flex', flex_flow='column', width='70%', background_color='#f0f0f0', padding='20px', border='2px solid #ccc', border_radius='10px')
  vbox = widgets.VBox([return_button, logo,  most_Freq, less_Freq, most_search_Freq,input_text,results_text], layout=vbox_layout)
  container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', height='100vh')
  container = widgets.Box(children=[vbox], layout=container_layout)

  # Display the elements
  display(container)

##Search Screen (main):

In [59]:

def Main_Screen():
    # Button actions
    def statistics_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_statistics_screen()

    def admin_screen_action(b):
        container.children=[]
        container.layout=widgets.Layout()
        create_admin_screen()
    # Create buttons with actions
    admin_screen_button = widgets.Button(description="Admin Screen", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
    statistics_button = widgets.Button(description="Statistics", layout=widgets.Layout(width='auto', margin='0 10px 0 0'))
    admin_screen_button.on_click(admin_screen_action)
    statistics_button.on_click(statistics_action)

    # Adjust layout for input field, button, and results output
    input_layout = widgets.Layout(width='50%', justify_content='center')
    button_layout = widgets.Layout(width='50%', justify_content='center', margin='10px 0 0 0')
    results_layout = widgets.Layout(border='1px solid black', border_radius='5px', width='50%', margin='20px 0', padding='10px', visibility='hidden')

    # Create the search text box, button, and results output
    search_input = widgets.Text(value='', placeholder='Type here to search', description='', disabled=False, layout=input_layout)
    search_button = widgets.Button(description='SEARCH', button_style='success', tooltip='Click to search', icon='search', layout=button_layout)
    results_output = widgets.Textarea(layout=results_layout)


    # Function to simulate a search operation and display results
    def perform_search(query):
      results_output.layout.visibility = 'visible'
      if search_input.value == None or search_input.value == "":
        results_output.value = ''
        return
      resultsLinks = search(search_input.value)
      results_output.value = '\n'.join(resultsLinks)

    search_button.on_click(perform_search)

    # Display logo as a widget
    with open("/content/logo.jpg", "rb") as file:  # Update the path to your logo
        image = file.read()
    logo = widgets.Image(value=image, format='jpg', layout=widgets.Layout(height='100px', width='auto', margin = '0 0 20px 0'))

    # Organize the layout
    top_buttons = widgets.HBox([admin_screen_button, statistics_button], layout=widgets.Layout(justify_content='flex-start', width='100%'))
    vbox_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', width='70%', background_color='#f0f0f0', padding='20px', border='2px solid #ccc', border_radius='10px')
    vbox = widgets.VBox([top_buttons, logo, search_input, search_button, results_output], layout=vbox_layout)
    container_layout = widgets.Layout(display='flex', flex_flow='column', align_items='center', height='100vh')
    container = widgets.Box(children=[vbox], layout=container_layout)

    # Display the elements
    display(container)


# **The Website**

In [60]:
Main_Screen()

Box(children=(VBox(children=(HBox(children=(Button(description='Admin Screen', layout=Layout(margin='0 10px 0 …

[[['https://azure.microsoft.com/en-us/free/', 1], ['https://azure.microsoft.com/en-us/products/data-factory/', 1], ['https://azure.microsoft.com/en-us/products/virtual-machines/dedicated-host/', 3]]]
[[['https://azure.microsoft.com/en-us/free/', 1], ['https://azure.microsoft.com/en-us/products/data-factory/', 1], ['https://azure.microsoft.com/en-us/products/virtual-machines/dedicated-host/', 3]], [['https://azure.microsoft.com/en-us/', 329], ['https://portal.azure.com/', 1], ['https://azure.microsoft.com/en-us/free/', 323], ['https://azure.microsoft.com/en-us/contact/', 257], ['https://azure.microsoft.com/en-us/explore/', 278], ['https://azure.microsoft.com/en-us/explore/global-infrastructure/', 296], ['https://azure.microsoft.com/en-us/solutions/cloud-economics/', 276], ['https://azure.microsoft.com/en-us/solutions/cloud-enablement/', 290], ['https://azure.microsoft.com/en-us/case-studies/', 250], ['https://azure.microsoft.com/en-us/products/', 485], ['https://azure.microsoft.com/en-u